In [ ]:
import numpy as np

# Exercise 2
Throughout this exercise we will use the following labelling of legs:
![title](labelling.png)

We will also be using the convention that we include trivial (i.e., one-dimensional) "phantom" singular values at either end of the MPS chain.

## 2.1

The first step is to implement functions that, given a MPS representation $M = [M^{[1]\sigma_1} , . . . ,M^{[N]\sigma_N}]$, compute the left-normalized
representation $[A^{[1]}, . . . ,A^{[n]}]$ and the right-normalized representation $[B^{[1]}, . . . ,B^{[n]}]$.

In [23]:
#what we have to do:
# reshape the Tensor into a matrix 
#take the tensor and plit it in U, S, Vh
#Vh absorbs the S
#U is the new A but we convert it back to a tensor

def left_normalize(Ms):
    """
    Convert a MPS to a left-normalized MPS.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    As : list of rank 3 tensors.
        A left-normalized MPS representation of the state.
    """

    As = []
    T = np.ones((1,1)) #initial tensor only has one element
    for M in Ms:
        np.tensordot(T,M,axes = (1,1)) #correct the axes                                            ########WHY DO THAT?
        np.transpose(M, [1,0,2]) #transpose the tensor so it has the right shape for the SVD
        d, chi1, chi2 = M.shape #get the dimensions of the tensor
        M = np.reshape(M, (d*chi1, chi2)) #reshape the tensor into a matrix to perform the SVD
        U, S, Vh = np.linalg.svd(M,full_matrices = False)
        As.append(np.reshape(U, (d, chi1, -1))) #convert the matrix back into a tensor
        T = np.matmul(np.diag(S), Vh) #absorb the S into Vh
        
    # Keep leftover signs (but no normalization)
    As[0] = As[0]*np.sign(T)                                                                          ########WHY DO THAT?
    return As

def right_normalize(Ms):
    """
    Convert a MPS to a right-normalized MPS.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    Bs : list of rank 3 tensors.
        A right-normalized MPS representation of the state.
    """

    Bs = []
    T = np.ones((1,1)) #initial tensor only has one element
    for M in reversed(Ms):
        np.tensordot(M,T,axes = (2,0))
        d, chi1, chi2 = M.shape
        M = np.transpose(M, [1,0,2])
        M = np.reshape(M, (chi1, d*chi2))
        U, S, Vh = np.linalg.svd(M,full_matrices = False)
        Vh = np.reshape(Vh, (-1, d, chi2))
        Bs.append(np.transpose(Vh, [1,0,2]))
        T = np.matmul(U, np.diag(S))

    Bs = Bs[::-1]
    Bs[0] = Bs[0]*np.sign(T)                                                                          ########WHY DO THAT?
    return Bs


## 2.2

Implement a function that constructs the Vidal canonical form and returns the matrices $\Gamma = [\Gamma^{[1]\sigma_1} , . . . ,\Gamma^{[N]\sigma_N}]$ and $\Lambda = [\Lambda^{[1]} , . . . ,\Lambda^{[N-1]}]$.

In [25]:
def truncate(U, S, Vh):
    """
    Remove singular values below threshold.
    """

    l = np.sum(np.abs(S) > 1e-8)
    return U[:, :l], S[:l], Vh[:l, :]

def vidal_form(Ms):
    """
    Convert a MPS to Vidal canonical form.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    Lambdas : list of rank 1 tensors.
        The Schmidt values of every site.

    Gammas : list of rank 3 tensors.
        The Gamma matrices in the Vidal canonical form.
    """

    Lambdas = [np.array([1])]
    Gammas = []
    Bs = right_normalize(Ms)
    T = np.ones((1,1))
    N = len(Bs)
    for i in range(N):
        Bi = np.tensordot(T, Bs[i], axes = (1,1))
        Bi = np.transpose(Bi, [1,0,2])
        d, chi1, chi2 = Bi.shape
        Bi = np.reshape(Bi, [d*chi1, chi2])
        U, S, Vh = truncate(*np.linalg.svd(Bi, full_matrices = False))
        
        A = np.reshape(U, [d, chi1, len(S)])
        Gamma = np.tensordot(np.diag(1.0/Lambdas[-1]), A, axes=(1, 1))
        Gammas.append(Gamma.transpose([1, 0, 2]))
        Lambdas.append(S)
        T = np.matmul(np.diag(S), Vh)


    return Lambdas, Gammas

You can use the implemented functions to get the Vidal's canonical form for states $|\Psi_1>$ and $|\Psi_2>$. 

In [34]:
def phi1(N):
    """
    Get MPS form of phi1.

    Parameters
    ----------
    N : int
        The number of sites.

    Returns
    -------
    Lambdas
        The Schmidt values of phi1.

    Gammas
        The Gamma matrices in the Vidal canonical form of phi1.
    """

    M0 = np.zeros((2, 1, 2))
    M0[0,:,:] = [1, 0]/np.sqrt(2)
    M0[1,:,:] = [0, 1]/np.sqrt(2)
    if N % 2 == 0:
        MN = np.zeros((2, 2, 1)) #MN is actually the matrix at site N-1
        MN[0,:,:] = np.vstack([0, 1]) 
        MN[1,:,:] = np.vstack([1, 0])
    else:
        MN = np.zeros((2, 2, 1))
        MN[0, :] = np.vstack([1, 0])
        MN[1, :] = np.vstack([0, 1])

    M_even = np.zeros((2, 2, 2))
    M_even[0,:,:] = [[1, 0], [0, 0]]
    M_even[1,:,:] = [[0, 0], [0, 1]]
    
    M_odd = np.flip(M_even, 0)

    Ms = [M0]
    for i in range(1, N - 1):
        if i % 2 == 0:
            Ms.append(np.copy(M_even))
        else:
            Ms.append(np.copy(M_odd))

    Ms.append(MN)
    return vidal_form(Ms)

def phi2(N):
    """
    Get MPS form of phi2.

    Parameters
    ----------
    N : int
        The number of sites.

    Returns
    -------
    Lambdas
        The Schmidt values of phi2.
        
    Gammas
        The Gamma matrices in the Vidal canonical form of phi2.
    """

    M0 = np.zeros((2, 1, 2))
    M0[0, :] = [1, 0]
    M0[1, :] = [1, 0]
    
    M = np.zeros((2, 2, 2))
    M[0] = np.identity(2)
    M[1] = np.identity(2)
    
    MN = np.transpose(M0, [0, 2, 1])

    Ms = [M0/np.sqrt(2)]
    for i in range(N - 2):
        Ms.append(np.copy(M)/np.sqrt(2))

    Ms.append(MN/np.sqrt(2))
    return vidal_form(Ms)

## 2.4

In [35]:
N = 10
Lambdas1, Gammas1 = phi1(N)
Lambdas2, Gammas2 = phi2(N)
print(Gammas1[7]) #up to this point everything is fine
print(Gammas1[8]) #here something not clear happens. The matrices at
#this site simply are not the ones one would expect (according
#to my understanding the rows are exchanged for some reason)
print(Gammas1[9]) #it should be with the two gammas reversed or rows
# exchanged 

[[[0.         0.        ]
  [0.         1.41421356]]

 [[1.41421356 0.        ]
  [0.         0.        ]]]
[[[1.41421356 0.        ]
  [0.         0.        ]]

 [[0.         0.        ]
  [0.         1.41421356]]]
[[[1.]
  [0.]]

 [[0.]
  [1.]]]


Write a function that, given two generic states in the Vidal's canonical form, evaluates their overlap as describe in the exercise sheet. Then, calculate the overlap between $|\Psi_1>$ and $|\Psi_2>$ for N = 30 spins.

In [43]:
def overlap(Lambdas1, Gammas1, Lambdas2, Gammas2):
    """
    Compute the overlap of two wave functions given in Vidal canonical form.

    Parameters
    ----------
    Lambdas1 : list of rank 1 tensors.
        Schmidt values of the first state.

    Gammas1 : list of rank 3 tensors.
        Gamma matrices of the first state.

    Lambdas2 : list of rank 1 tensors.
        Schmidt values of the second state.

    Gammas2 : list of rank 3 tensors.
        Gamma matrices of the second state.

    Returns
    -------
    complex
        Overlap of the two states
    """
    #print(len(Gammas1))
    #print(len(Gammas2))
    overlap = np.tensordot(Gammas2[0], Gammas1[0].conj(), axes = (0,0))
    overlap = np.transpose(overlap, [0,2,1,3])
    #print(overlap.shape)
    for i in range(1, len(Gammas1)):
        overlap = np.tensordot(overlap, np.diag(Lambdas2[i]), axes = (2,0))
        overlap = np.tensordot(overlap, np.diag(Lambdas1[i]), axes = (2,0))
        
        overlap = np.tensordot(overlap, Gammas2[i], axes = (2,1))
        overlap = np.tensordot(overlap, Gammas1[i].conj(), axes =([2,3], [1, 0]))                    #how to understand what values to se in the axes?
        #print(overlap.shape)
        
    return overlap.flatten()[0]

In [45]:
N = 30
Lambdas1, Gammas1 = phi1(N)
Lambdas2, Gammas2 = phi2(N)
print("Numeric:", overlap(Lambdas1, Gammas1, Lambdas2, Gammas2))
print("Analytical:", 2/np.sqrt(2)/(np.sqrt(2))**N)

Numeric: -4.315837287515538e-05
Analytical: 4.31583728751554e-05


Verify the correct implementation of your function by calculating the normalization of an arbitrary MPS state in canonical form (should be equal to 1).

In [46]:
def random_MPS(d, N):
    """
    returns random MPS with bond dimension d
    """
    A1 = np.random.rand(2,1,d) #starting points only zeilevecotrs aber d mal
    Ms = [A1]
    for i in range(N - 2):
        Ai=np.random.rand(2,d,d) #adding random matrices for both cases
        Ms.append(Ai)

    An = np.random.rand(2,d,1) #last point only spaltenvektor
    Ms.append(An)
    return Ms

def check_left_normalisation(A):
    """
    returns True, if A is right-normalized. 
    Only yields true result if rank of singular values not smaller than bond dimension.
    has to be modified!
    """
    E = np.tensordot(A, np.conjugate(A), axes=([0, 1], [0, 1]))
    return np.linalg.norm(E - np.eye(np.shape(E)[0])) < 1e-4

def check_right_normalisation(A):
    """
    returns True, if A is right-normalized. 
    Only yields true result if rank of singular values not smaller than bond dimension.
    has to be modified!
    """
    E = np.tensordot(A, np.conjugate(A), axes=([0, 2], [0, 2]))
    return np.linalg.norm(E - np.eye(np.shape(E)[0])) < 1e-4

def check_properties(Lambdas, Gammas):
    properties = True
    for i in range(len(Gammas)):
        A = np.tensordot(np.diag(Lambdas[i]), Gammas[i], axes=(1, 1))
        A = np.transpose(A, [1, 0, 2])
        B = np.tensordot(Gammas[i], np.diag(Lambdas[i+1]), axes=(2, 0))
        properties = properties and check_left_normalisation(A) and check_right_normalisation(B)

    return properties

Lambdas_rand, Gammas_rand = vidal_form(random_MPS(2, 30))
print("Norm:", overlap(Lambdas_rand, Gammas_rand, Lambdas_rand, Gammas_rand))
print("Is canonical:", check_properties(Lambdas_rand, Gammas_rand))

Norm: 1.0000000000000009
Is canonical: True
